In [426]:
# [시험용 데이터셋 만들기]
# (단, y_test 데이터는 활용해서는 안됨)
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=2021)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[id_name, target])
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[id_name, target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("tutorial.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, 
                                                  target='Reached.on.Time_Y.N', 
                                                  id_name='ID')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

#warehouse_block = > 지워, Mode_of_Shipment = > 사용해, Product_importance 사용, Gender 지워  = > object,  Customer_care_calls =>지워
#Customer_rating지워
y_test_id = pd.DataFrame(y_test["ID"])
data = pd.concat([X_train, y_train], axis = 1)
X_train.drop(columns = ["Warehouse_block",'Gender','Customer_care_calls','Customer_rating',"Weight_in_gms"],inplace = True)
X_test.drop(columns = ["Warehouse_block",'Gender','Customer_care_calls','Customer_rating',"Weight_in_gms"],inplace = True)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_train["Mode_of_Shipment"] = le.fit_transform(X_train["Mode_of_Shipment"])
X_train["Product_importance"] = le.fit_transform(X_train["Product_importance"])
X_test["Mode_of_Shipment"] = le.fit_transform(X_test["Mode_of_Shipment"])
X_test["Product_importance"] = le.fit_transform(X_test["Product_importance"])

#연관관계 분석  data.corr()
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST , Y_TRAIN, Y_TEST = train_test_split(X_train, y_train['Reached.on.Time_Y.N'], test_size  = 0.2, random_state = 10)

from sklearn.ensemble import RandomForestClassifier

#best 모델찾기
# model = RandomForestClassifier(n_estimators=200,max_depth = 10)
# model.fit(X_TRAIN, Y_TRAIN)
# Y_TEST_PREDICTED = model.predict(X_TEST)

model= RandomForestClassifier(n_estimators = 200, max_depth = 10)
model.fit(X_train, y_train['Reached.on.Time_Y.N'])
y_test_predicted = model.predict(X_test)

submit = pd.DataFrame({
    "ID" : y_test['ID'],
    'Reached.on.Time_Y.N' : y_test_predicted
})

submit.to_csv("submit.csv",index = False)